Project 2 
ENPM 661 Planning for Autonomous Robots 



Heqin Wang
UID:115489691 
Complete alone without a group member

In [ ]:
import numpy as np
import math
import heapq
import time
import matplotlib.pyplot as plt
import cv2
import pygame

# Record the start time to measure execution duration
start_time = time.time()


In [ ]:
class PointRobot:
    def __init__(self, position):
        self.position = position

    def move(self, direction):
        x, y = self.position
        moves = {
            "UP": (x, y-1),
            "DOWN": (x, y+1),
            "LEFT": (x-1, y),
            "RIGHT": (x+1, y),
            "UP_LEFT": (x-1, y-1),
            "UP_RIGHT": (x+1, y-1),
            "DOWN_LEFT": (x-1, y+1),
            "DOWN_RIGHT": (x+1, y+1),
        }
        if direction in moves and self.is_valid_move(moves[direction]):
            self.position = moves[direction]
            return self.position, True
        return self.position, False

    def is_valid_move(self, position):
        x, y = position
        return 0 <= x < 300 and 0 <= y < 200


In [ ]:
# Start the robot at a starting position
robot = PointRobot((0, 0))

# Example movements
movements = ["UP", "RIGHT", "DOWN_RIGHT"]
for move in movements:
    new_position, success = robot.move(move)
    if success:
        print(f"Moved {move} to {new_position}")
    else:
        print(f"Failed to move {move} from {robot.position}")


In [ ]:
# Simplify the creation of grid points
all_points = [(x, y) for x in range(301) for y in range(201)]


In [ ]:
class GridGraph:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.graph = self.initialize_graph()
    
    def initialize_graph(self):
        graph = {}
        for x in range(self.width):
            for y in range(self.height):
                neighbors = self.find_neighbors(x, y)
                graph[(x, y)] = neighbors
        return graph
    
    def find_neighbors(self, x, y):
        directions = [(0, 1), (1, 0), (-1, 0), (0, -1),
                      (1, 1), (-1, -1), (1, -1), (-1, 1)]
        neighbors = []
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.width and 0 <= ny < self.height:
                neighbors.append((nx, ny))
        return neighbors


In [ ]:
    def calculate_costs(self):
        costs = {}
        for node, neighbors in self.graph.items():
            costs[node] = {}
            for neighbor in neighbors:
                if self.is_diagonal_move(node, neighbor):
                    costs[node][neighbor] = 1.414
                else:
                    costs[node][neighbor] = 1
        return costs
    
    @staticmethod
    def is_diagonal_move(node, neighbor):
        return abs(node[0] - neighbor[0]) == 1 and abs(node[1] - neighbor[1]) == 1


In [ ]:
class DijkstraPathFinder:
    def __init__(self, graph, start):
        self.graph = graph
        self.start = start
        self.all_distance = {vertex: math.inf for vertex in self.graph}
        self.all_distance[start] = 0
        self.backtracking = {}
        self.visited = set()
        self.priority_queue = [(0, start)]

    def search(self, goal):
        while self.priority_queue:
            current_distance, current_vertex = heapq.heappop(self.priority_queue)

            if current_vertex == goal:
                print('GOAL REACHED')
                return self.all_distance, self.backtracking

            if current_distance > self.all_distance[current_vertex]:
                continue

            for neighbor, cost in self.graph[current_vertex].items():
                new_distance = current_distance + cost
                if new_distance < self.all_distance[neighbor]:
                    self.all_distance[neighbor] = new_distance
                    heapq.heappush(self.priority_queue, (new_distance, neighbor))
                    self.backtracking[neighbor] = current_vertex
                    self.visited.add(current_vertex)

        return self.all_distance, self.backtracking


In [ ]:
    def backtrack_path(self, goal):
        path = [goal]
        while path[-1] != self.start:
            path.append(self.backtracking[path[-1]])
        path.reverse()
        return path


In [ ]:
# Assume 'graph' is previously defined as a dictionary where keys are nodes and values are dictionaries
# of neighboring nodes with their corresponding edge costs.
start_node = (0, 0)
goal_node = (50, 50)
path_finder = DijkstraPathFinder(graph, start_node)

# Perform the search for the path to the goal
_, backtracking = path_finder.search(goal_node)

# Backtrack to find the path from start to goal
path = path_finder.backtrack_path(goal_node)

print("Path from start to goal:", path)


In [ ]:
def collect_user_input():
    x_start = int(input("Enter the x coordinate of the start: "))
    y_start = int(input("Enter the y coordinate of the start: "))
    x_goal = int(input("Enter the x coordinate of the goal: "))
    y_goal = int(input("Enter the y coordinate of the goal: "))
    return (x_start, y_start), (x_goal, y_goal)


In [ ]:
def execute_pathfinding():
    start, goal = collect_user_input()
    width, height = 300, 200
    # Assuming PointRobotdijkstra is refactored to a class that encapsulates the given logic
    pathfinder = PointRobotPathfinder(width, height, start, goal)
    pathfinder.find_obstacles()
    pathfinder.generate_graph()
    pathfinder.execute_dijkstra()
    
    # Visualize the result
    pathfinder.visualize_path()
    # Animate the path discovery and backtracking
    pathfinder.animate_discovery()

    print("Final Path:", pathfinder.backtrack_path())
    print("Total Time Taken : ", time.time() - start_time, "seconds")


In [ ]:
class PointRobotPathfinder:
    # Initialization and method definitions as before
    
    def visualize_path(self):
        # Assuming `new_canvas` is part of this class now
        for point in self.list_of_all_points:
            x, y = point[1], point[0]
            self.new_canvas[(y, x)] = [0, 255, 255]  # Yellow for obstacles
        self.new_canvas = np.flipud(self.new_canvas)
        cv2.imshow('Path', cv2.resize(self.new_canvas, (600, 400)))
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def animate_discovery(self):
        # Animation logic integrated here
        pygame.init()
        gameDisplay = pygame.display.set_mode((self.width, self.height))
        pygame.display.set_caption('Path Discovery Animation')
        # Animation loop as before


In [ ]:
if __name__ == "__main__":
    execute_pathfinding()
